# LASSO Python
## MBPP Evaluation Analytics
This notebook showcases how a data analyst can use LASSO Python to collect dynamic information about Python code implementations.
By using data from an Apache Ignite cache that was filled by LASSO Python, the analyst can easily deduce interesting information, such as correctness of the code, execution times, or coverage data.
Both the tests and the Python code implementations used in this example stem from the sanitized Mostly Basic Python Problems (MBPP) dataset. MBPP is a dataset consisting of crowd-sourced Python programming problems. Each task consists of a description, a code solution and multiple test cases. By using LASSO Python, these tests were replicated with the results shown in this notebook. More information on MBPP can be found here: https://github.com/google-research/google-research/tree/master/mbpp

In [49]:
# Import result csv that was generated by LASSO Python and extracted from the Apache Ignite cache
import pandas as pd

file_path = './evaluation_results.csv'
df = pd.read_csv(file_path)
number_of_tasks = df['ABSTRACTIONID'].nunique()
print(f'Number of tasks: {number_of_tasks}')
df.head()

Number of tasks: 399


,EXECUTIONID,ABSTRACTIONID,ACTIONID,ARENAID,SHEETID,SYSTEMID,VARIANTID,ADAPTERID,X,Y,TYPE,VALUE,RAWVALUE,VALUETYPE,LASTMODIFIED,EXECUTIONTIME
0,4930e080-8e3e-4456-8728-58abf2f88c88,Task57,NaN,execute,57.xlsx,6f24ff5d-080f-48ea-bb49-45d447f5ae33,original,"0('find_Max_Num', 'find_Max_Num', 0)",2,2,input_value,-,-,<class 'str'>,"(datetime.datetime(2024, 9, 14, 11, 31, 4, 160...",-1
1,4930e080-8e3e-4456-8728-58abf2f88c88,Task750,NaN,execute,750.xlsx,NaN,original,"0('create', 'createPythonObject', 0)",6,1,input_value,9,9,<class 'int'>,"(datetime.datetime(2024, 9, 14, 11, 43, 35, 53...",-1
2,4930e080-8e3e-4456-8728-58abf2f88c88,Task422,NaN,execute,422.xlsx,5ba4efaa-1f01-4c0d-aad8-044acc88b516,original,"0('find_Average_Of_Cube', 'find_Average_Of_Cub...",-1,2,metrics_covered_branches_in_file,2,2,<class 'int'>,"(datetime.datetime(2024, 9, 14, 11, 37, 46, 72...",-1
3,4930e080-8e3e-4456-8728-58abf2f88c88,Task569,NaN,execute,569.xlsx,NaN,original,"0('create', 'createPythonObject', 0)",3,30,input_value,"['a', 'b']","['a', 'b']",<class 'list'>,"(datetime.datetime(2024, 9, 14, 11, 40, 18, 53...",-1
4,4930e080-8e3e-4456-8728-58abf2f88c88,Task752,NaN,execute,752.xlsx,d44d0cff-abdf-4b83-969f-755c12608773,original,"0('jacobsthal_num', 'jacobsthal_num', 0)",0,2,value,1,1,<class 'int'>,"(datetime.datetime(2024, 9, 14, 11, 43, 41, 46...",56


In [50]:
# Compare oracle values (ground truth) with the values resulting from the LASSO Python test execution
value_df = df.query('TYPE == "value"')
oracle_df = df.query('TYPE == "oracle"')

merged_df = pd.merge(value_df, oracle_df, on=['EXECUTIONID', 'ABSTRACTIONID', 'SHEETID', 'X', 'Y'])
merged_df[['ABSTRACTIONID', 'Y', 'TYPE_x', 'VALUE_x', 'TYPE_y', 'VALUE_y']]

,ABSTRACTIONID,Y,TYPE_x,VALUE_x,TYPE_y,VALUE_y
0,Task752,2,value,1,oracle,1
1,Task56,2,value,False,oracle,False
2,Task413,14,value,"[99, 96, 94, 98]",oracle,"[99, 96, 94, 98]"
3,Task751,2,value,True,oracle,True
4,Task644,6,value,"[5, 4, 6, 7]",oracle,"[5, 4, 6, 7]"
...,...,...,...,...,...,...
1211,Task730,6,value,"[10, 15, 19, 18, 17, 26, 17, 1",oracle,"[10, 15, 19, 18, 17, 26, 17, 1"
1212,Task265,12,value,"[[1, 4, 7, 10, 13], [2, 5, 8,",oracle,"[[1, 4, 7, 10, 13], [2, 5, 8,"
1213,Task763,6,value,4,oracle,4
1214,Task457,18,value,['x'],oracle,['x']


In [51]:
# Cases were return value and oracle value are equal
successful_test_runs = merged_df.query('VALUE_x == VALUE_y')['ABSTRACTIONID'].nunique()
print(f"{successful_test_runs}/{number_of_tasks} tasks with equal return and oracle values ({successful_test_runs/number_of_tasks*100:.2f}%)")
merged_df.query('VALUE_x == VALUE_y')[['ABSTRACTIONID', 'Y', 'TYPE_x', 'VALUE_x', 'TYPE_y', 'VALUE_y']]

362/399 tasks with equal return and oracle values (90.73%)


,ABSTRACTIONID,Y,TYPE_x,VALUE_x,TYPE_y,VALUE_y
0,Task752,2,value,1,oracle,1
1,Task56,2,value,False,oracle,False
2,Task413,14,value,"[99, 96, 94, 98]",oracle,"[99, 96, 94, 98]"
3,Task751,2,value,True,oracle,True
4,Task644,6,value,"[5, 4, 6, 7]",oracle,"[5, 4, 6, 7]"
...,...,...,...,...,...,...
1211,Task730,6,value,"[10, 15, 19, 18, 17, 26, 17, 1",oracle,"[10, 15, 19, 18, 17, 26, 17, 1"
1212,Task265,12,value,"[[1, 4, 7, 10, 13], [2, 5, 8,",oracle,"[[1, 4, 7, 10, 13], [2, 5, 8,"
1213,Task763,6,value,4,oracle,4
1214,Task457,18,value,['x'],oracle,['x']


In [52]:
# Cases were return value and oracle value are different
different_return_and_oracle = merged_df.query('VALUE_x != VALUE_y')['ABSTRACTIONID'].nunique()
print(f"{different_return_and_oracle}/{number_of_tasks} tasks with different return and oracle values ({different_return_and_oracle/number_of_tasks*100:.2f}%)")
merged_df.query('VALUE_x != VALUE_y')[['ABSTRACTIONID', 'Y', 'TYPE_x', 'VALUE_x', 'TYPE_y', 'VALUE_y']]

52/399 tasks with different return and oracle values (13.03%)


,ABSTRACTIONID,Y,TYPE_x,VALUE_x,TYPE_y,VALUE_y
15,Task126,3,value,NaN,oracle,3
26,Task419,2,value,UNSUCCESSFUL,oracle,243
45,Task117,33,value,"[(6.0, 78.0), (5.0, 26.45), (1",oracle,"[(6, 78), (5, 26.45), (1.33, 4"
46,Task391,39,value,UNSUCCESSFUL,oracle,"[{None: {'java': 10}}, None, N"
59,Task267,3,value,120,oracle,84
...,...,...,...,...,...,...
1167,Task622,3,value,16.0,oracle,16
1169,Task412,9,value,UNSUCCESSFUL,oracle,"[10, 20]"
1171,Task744,2,value,False,oracle,True
1189,Task12,9,value,UNSUCCESSFUL,oracle,"[[1, 1, 1], [1, 2, 3], [2, 4,"


In [53]:
# Cases were the return value and oracle value are different but the was not unsuccessful
print(merged_df.query('VALUE_x != "UNSUCCESSFUL" and VALUE_x != VALUE_y')['ABSTRACTIONID'].nunique(), f"/ {number_of_tasks} tasks without errors where oracle and return values are different")
merged_df.query('VALUE_x != "UNSUCCESSFUL" and VALUE_x != VALUE_y')[['ABSTRACTIONID', 'Y', 'TYPE_x', 'VALUE_x', 'TYPE_y', 'VALUE_y']]

28 / 399 tasks without errors where oracle and return values are different


,ABSTRACTIONID,Y,TYPE_x,VALUE_x,TYPE_y,VALUE_y
15,Task126,3,value,NaN,oracle,3
45,Task117,33,value,"[(6.0, 78.0), (5.0, 26.45), (1",oracle,"[(6, 78), (5, 26.45), (1.33, 4"
59,Task267,3,value,120,oracle,84
68,Task432,2,value,15.0,oracle,15
88,Task628,2,value,I_am_a_Programmer,oracle,I%20am%20a%20Programmer
...,...,...,...,...,...,...
1145,Task14,1,value,240.0,oracle,240
1157,Task554,9,value,[8],oracle,"[7, 9, 1]"
1163,Task628,3,value,I_love_Coding,oracle,I%20love%20Coding
1167,Task622,3,value,16.0,oracle,16


In [54]:
# Cases were the execution was unsuccessful
print(merged_df.query('VALUE_x == "UNSUCCESSFUL"')['ABSTRACTIONID'].nunique(), "tasks that (partially) failed")
merged_df.query('VALUE_x == "UNSUCCESSFUL"')[['ABSTRACTIONID', 'Y', 'TYPE_x', 'VALUE_x', 'TYPE_y', 'VALUE_y']]

24 tasks that (partially) failed


,ABSTRACTIONID,Y,TYPE_x,VALUE_x,TYPE_y,VALUE_y
26,Task419,2,value,UNSUCCESSFUL,oracle,243
46,Task391,39,value,UNSUCCESSFUL,oracle,"[{None: {'java': 10}}, None, N"
74,Task779,34,value,UNSUCCESSFUL,oracle,"{(1, 2): 1, (3, 4): 1, (4, 5):"
84,Task133,2,value,UNSUCCESSFUL,oracle,-32
87,Task723,6,value,UNSUCCESSFUL,oracle,11
...,...,...,...,...,...,...
1134,Task129,6,value,UNSUCCESSFUL,oracle,True
1160,Task614,5,value,UNSUCCESSFUL,oracle,30
1169,Task412,9,value,UNSUCCESSFUL,oracle,"[10, 20]"
1189,Task12,9,value,UNSUCCESSFUL,oracle,"[[1, 1, 1], [1, 2, 3], [2, 4,"


In [55]:
print("Task identifiers of tasks that (partially) failed:")
set(merged_df.query('VALUE_x == "UNSUCCESSFUL"')[['ABSTRACTIONID', 'Y', 'TYPE_x', 'VALUE_x', 'TYPE_y', 'VALUE_y']]["ABSTRACTIONID"])

Task identifiers of tasks that (partially) failed:


{'Task105',
 'Task12',
 'Task123',
 'Task129',
 'Task133',
 'Task142',
 'Task172',
 'Task295',
 'Task391',
 'Task398',
 'Task412',
 'Task419',
 'Task558',
 'Task614',
 'Task615',
 'Task627',
 'Task723',
 'Task724',
 'Task735',
 'Task757',
 'Task758',
 'Task779',
 'Task799',
 'Task805'}

In [56]:
# Obtain statistics for the coverage metrics
coverage_df = df.query('TYPE == "metrics_covered_lines_in_function_ratio"')

coverage_df['VALUE'].astype(float).describe()

count    1161.000000
mean       92.740134
std        14.614139
min         8.823529
25%        93.548387
50%       100.000000
75%       100.000000
max       100.000000
Name: VALUE, dtype: float64

In [57]:
# Obtain information about the longest execution times
sorted_df = df.sort_values(by='EXECUTIONTIME', ascending=False)
sorted_df.head()

,EXECUTIONID,ABSTRACTIONID,ACTIONID,ARENAID,SHEETID,SYSTEMID,VARIANTID,ADAPTERID,X,Y,TYPE,VALUE,RAWVALUE,VALUETYPE,LASTMODIFIED,EXECUTIONTIME
10616,4930e080-8e3e-4456-8728-58abf2f88c88,Task583,NaN,execute,583.xlsx,c2f68315-be8e-4564-9cbb-68e65ee2d68e,original,"0('catalan_number', 'catalan_number', 0)",0,1,value,16796,16796,<class 'int'>,"(datetime.datetime(2024, 9, 14, 11, 40, 44, 29...",21027
30133,4930e080-8e3e-4456-8728-58abf2f88c88,Task583,NaN,execute,583.xlsx,c2f68315-be8e-4564-9cbb-68e65ee2d68e,original,"0('catalan_number', 'catalan_number', 0)",1,1,op,catalan_number,catalan_number,function,"(datetime.datetime(2024, 9, 14, 11, 40, 44, 29...",21027
6267,4930e080-8e3e-4456-8728-58abf2f88c88,Task583,NaN,execute,583.xlsx,c2f68315-be8e-4564-9cbb-68e65ee2d68e,original,"0('catalan_number', 'catalan_number', 0)",0,2,value,4862,4862,<class 'int'>,"(datetime.datetime(2024, 9, 14, 11, 40, 44, 29...",6383
14529,4930e080-8e3e-4456-8728-58abf2f88c88,Task583,NaN,execute,583.xlsx,c2f68315-be8e-4564-9cbb-68e65ee2d68e,original,"0('catalan_number', 'catalan_number', 0)",1,2,op,catalan_number,catalan_number,function,"(datetime.datetime(2024, 9, 14, 11, 40, 44, 29...",6383
23373,4930e080-8e3e-4456-8728-58abf2f88c88,Task84,NaN,execute,84.xlsx,3e66bdd7-bc93-4f0e-ac3b-5eb45bec5127,original,"0('sequence', 'sequence', 0)",0,1,value,6,6,<class 'int'>,"(datetime.datetime(2024, 9, 14, 11, 31, 52, 68...",1015
